# Multi-Agent Pipeline Flow Summary Table

| **State** | **Agents Created** | **Agent Responsibilities** | **Unit Tests?** | **Debugging?** | **Outcome / Transition Trigger** |
|----------|--------------------|----------------------------|------------------|----------------|----------------------------------|
| **1. Understand Background** | Reader → Reviewer → Summarizer | Reader extracts competition background → Reviewer checks format → Summarizer finalizes | ❌ None | ❌ No | JSON extracted → Move to Preliminary EDA |
| **2. Preliminary EDA** | Planner → Developer → Reviewer → Summarizer | Planner creates EDA plan; Developer generates plots; Reviewer checks; Summarizer summarizes | ✔ Yes (image count) | ❌ No | All tests passed → Move to Data Cleaning |
| **3. Data Cleaning** | Planner → Developer → Reviewer → Summarizer | Planner outlines cleaning; Developer writes cleaning code; Unit tests enforce data integrity | ✔ Extensive tests | ✔ Yes (missing Fare fixed) | All tests passed → Move to In-depth EDA |
| **4. In-depth EDA** | Planner → Developer → Reviewer → Summarizer | Planner outlines deep EDA; Developer executes; Reviewer checks; Summarizer writes summary | ✔ Yes (image count) | ❌ No | All tests passed → Move to Feature Engineering |
| **5. Feature Engineering** | Planner → Developer → Reviewer → Summarizer | Planner defines FE; Developer executes transformations; Reviewer checks output; Summarizer writes summary | ✔ Yes (columns, ID, target, feature explosion) | ❌ No | All tests passed → Move to Model Building |
| **6. Model Building, Validation, Prediction** | Planner → Developer → Reviewer → Summarizer | Planner defines training; Developer trains model and produces submission.csv | ✔ Yes (submission format, column names, validity) | ✔ Yes (1st fail → debug → 2nd pass) | All tests passed → SOP completes |
| **7. SOP Completed** | — | — | — | — | **Competition titanic SOP is completed** |



# Implementation

The pipeline progresses by iterating through States (phases):

- **State phases:**
  - "Understand Background"
  - "Preliminary Exploratory Data Analysis"
  - "Data Cleaning"
  - "In-depth Exploratory Data Analysis"
  - "Feature Engineering"
  - "Model Building, Validation, and Prediction"

## Phase Execution

1. Create a memory directory for each phase State.
2. Iterate through each agent in that phase.
3. Each agent returns a memory dict like `{ "planner": { ... } }` and calls `update_memory(memory)` to merge output into the current phase memory.

## Memory Model

- Each phase gets its own memory (not automatically shared with later phases).
- Agents write to the phase memory dict, e.g.:

```json
{
  "reader": {
    "history": "...",
    "role": "Reader",
    "description": "...",
    "task": "...",
    "input": "<background_info>...</background_info>",
    "summary": "...",
    "result": "..."
  },
  "planner": {
    "history": "...",
    "role": "Planner",
    "description": "...",
    "task": "...",
    "input": "<background_info>...</background_info>",
    "summary": "...",
    "result": "..."
  },
  "reviewer": {
    "history": "...",
    "score": 3,
    "suggestion": "...",
    "result": "review text"
  }
}
```

## Iterative Repeats (Low Score)

When the phase score is low, the process repeats for a limited number of iterations: the score is determined the avarage of all the agents in the phase/ if developer did not run=0

```python
while not state.finished:
    agent = <planner/developer/reviewer/summarizer>
    agent.action(state)
    state.update_memory(...)
    state.next_step()

if state.finished:
    state.set_score()  # evaluate score

if state.score < 3:
    new_state = State(phase=old_phase)
    new_state.memory = deepcopy(old_memory)
```

- Repeat re-runs the same phase with carried memory plus a fresh slot.
- Success advances to the next phase and starts with a fresh memory unless explicitly copied.


## The **Planner**
1. Check if `state.memory` length > 1; evaluate score.
2. Execution (4 rounds):

   Round 0:
   - Read data files in the phase and create a data preview summary.
   - Set background info in state with the data preview.
   - Get state info: phase guidelines and goals (from State class).
   - Formulate the prompt so it is added to LLM history.

   Round 1:
   - Retrieve previous phases' PLAN and REPORT; read current data.
   - Use READ tools (needs modification for new toolsmith outputs).
   - Generate a new PLAN using previous plans, previous reports, current data, and tool documentation.

   Round 2:
   - Reorganize the plan into clean markdown structure.

   Round 3:
   - Convert final markdown plan into JSON schema (structured plan object).

   Tool integration:
   - Planner must reference tools generated by ToolSmith.
   - Controlled via `agent_base._get_tools` resolution logic.



## The **Agent_BASE**
- Initialization: Stores role, description, model, construct the LLM instnace from  OPENAI
- Memory/context helpers
_gather_experience_with_suggestion(state): aggregates past agent outputs, reviewer feedback, and (for developer) error/not-pass info from files.
- Data helpers: 
    - **NEEDUPDATE** _read_data(state, num_lines): reads sample rows from train/test/cleaned/processed CSVs depending on phase; extracts eval metric via LLM in model-building phase.
    - _data_preview(state, num_lines): asks LLM to produce a preview using PROMPT_DATA_PREVIEW; writes data_preview.txt.
    - **NEEDUPDATE** _get_feature_info(state): compares columns before/after a phase, infers target, returns a feature summary prompt string.
    - **NEEDUPDATE** Tool retrieval：  **_get_tools(state)** -> (tools_text, tool_names): Loads phase→tool names from config.json.
        - For developer in certain phases, extracts tool_names from the markdown plan via LLM; else uses config’s list.
        - Builds a vector DB with OpenaiEmbeddings + RetrieveTool and queries tool docs by name.
        - Returns a concatenated tools description string and the list of names; also writes tools_used_in_dir.md.
        - RetrieveTool Class: **NEEDUPDATE**, Which is used by developer and planner


## STATE.PY

- Loads config (agents per phase, directories, unit tests, rules, allowed tools).
- Resolves agents for the phase, directories, unit tests, rulebook parameters, and ml tool names.
- Sets competition_dir, dir_name, restore_dir, ml_tools, background_info, context.
- **NEEDUPDATE** get_state_info: phase-specific guidance text.
- **CANUSE** set_background_info, get_current_agent.
- generate_rules and _format_rules: writes user_rules.txt for the phase. **UPDATE CONFIG?**
- set_score: Get score from reviewer **Potential UPDATE**




        

# Plan

OVERVIEW: 
1. Run the background understanding agent to read the data challenge and inspect its output.
2. Call an agent to read that background output & Data and generate:
   - A domain‑specific tool (Python function/module)
   - A markdown file that documents the generated tool
3. Refer the Developer and Planner agent to the generated tools
    - Flow inside each phase: ToolSmith → Planner → Developer → Reviewer → Summarizer


TODOs 

**(config.json)**:
- Add  Toolsmith agent to config.json under phase_to_agents for the phases where it should run first.  
    - Update `phase_to_agents` to include the ToolSmith agent.
    - Update `phase_to_ml_tools` to point to generated tools.
    - Update `rulebook_parameters` to reflect generated tools.

**(STATE.PY)**:
- Ensure Toolsmith writes multi_agents/function_to_schema.json; then call state.reload_function_registry() before Developer/Planner use tools.
- Constructor Parameters: phase str, competiton str, message str
- Attributes: 
    - phase, competition, message
    - **memory:(List[Dict[str, Any]]): History of agent actions/results (starts with [{}])**; 
    - current_step int to track which agent; 
    - score: Set by the Reviewer Agent at the end of each state (Iterated through every agent)

**(Agent_BASE)**
- See above



#### Optionally generate unit tests to validate new tools.

